## 1. Configuração e Carregamento

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

# --- Configurações ---
# Vamos "descobrir" 2 fatores latentes (K=2).
# Por exemplo, o modelo pode descobrir que o Fator 1 é "Mainstream vs. Aventura"
# e o Fator 2 é "Cultura vs. Praia".
K = 2

print("Bibliotecas (pandas, numpy, sklearn) carregadas.")

Bibliotecas (pandas, numpy, sklearn) carregadas.


## 2. Carregar e Preparar os Dados (A Matriz)

In [2]:
# Carregar APENAS os ratings. Ignoramos os outros ficheiros.
try:
    df_ratings = pd.read_csv('../data/user_interactions_MVP.csv')
    print(f"Carregados {len(df_ratings)} ratings.")
except FileNotFoundError:
    print("ERRO: Ficheiro 'user_interactions_MVP.csv' não encontrado!")

# 1. Criar a Matriz Utilizador-Item
# Pivotar a tabela: utilizadores nas linhas, províncias nas colunas
# fillna(0) = Se um utilizador não visitou, o rating é 0.
df_matrix = df_ratings.pivot(
    index='user_id',
    columns='provincia_visitada',
    values='rating'
).fillna(0)

# 2. Converter para Matriz Esparsa (Obrigatório para SVD)
# O csr_matrix é uma forma eficiente de guardar esta matriz, que é maioritariamente zeros.
matrix_esparsa = csr_matrix(df_matrix.values)

print("\n--- Matriz Utilizador-Item (Input do ML) ---")
print(df_matrix)

Carregados 9 ratings.

--- Matriz Utilizador-Item (Input do ML) ---
provincia_visitada  Benguela  Cuando Cubango  Huambo  Huíla  Luanda  Malanje
user_id                                                                     
1                        4.0             0.0     0.0    2.0     5.0      0.0
2                        0.0             5.0     0.0    0.0     1.0      5.0
3                        1.0             0.0     5.0    0.0     0.0      4.0


## 3. Treinar o Modelo de ML (SVD)

In [3]:
# 1. Inicializar o Modelo SVD
model = TruncatedSVD(n_components=K, random_state=42)

# 2. Treinar o Modelo
# O SVD vai "aprender" os fatores latentes a partir da matriz esparsa
model.fit(matrix_esparsa)

print(f"\nModelo SVD (K={K}) treinado com sucesso.")

# 3. Extrair os Fatores Latentes (O que o modelo aprendeu)
# Estes são os "perfis" que o ML descobriu SOZINHO para os utilizadores
factores_user = model.transform(matrix_esparsa)

# Estes são os "perfis" que o ML descobriu SOZINHO para as províncias
factores_item = model.components_


Modelo SVD (K=2) treinado com sucesso.


## 4. Gerar Previsões (Reconstruir a Matriz)

In [4]:
# Multiplicar os dois fatores para obter a matriz de previsões completa
matriz_reconstruida_scores = np.dot(factores_user, factores_item)

# Converter de volta para um DataFrame legível
df_previsoes_svd = pd.DataFrame(
    matriz_reconstruida_scores, 
    index=df_matrix.index, 
    columns=df_matrix.columns
)

print("\n--- MATRIZ DE PREVISÕES (Scores do SVD) ---")
# Estes não são ratings de 1-5, são 'scores de afinidade'. Quanto mais alto, melhor.
print(df_previsoes_svd.round(2))


--- MATRIZ DE PREVISÕES (Scores do SVD) ---
provincia_visitada  Benguela  Cuando Cubango  Huambo  Huíla  Luanda  Malanje
user_id                                                                     
1                       4.00           -0.00   -0.00    2.0    5.00     -0.0
2                       0.49            3.05    2.44   -0.0    0.61      5.0
3                       0.39            2.44    1.95   -0.0    0.49      4.0


## 5. Gerar Recomendações (Top-K)

In [5]:
K_recs = 3 # O número de recomendações que queremos mostrar (igual ao K dos outros modelos)
recomendacoes_svd = {}

for user_id in df_previsoes_svd.index:
    # Obter os scores previstos para este utilizador
    user_scores = df_previsoes_svd.loc[user_id]
    
    # Ordenar do maior para o menor
    recs = user_scores.sort_values(ascending=False).index.tolist()
    
    # Remover itens que o utilizador JÁ AVALIOU (opcional, mas boa prática)
    # Neste sprint, vamos simplificar e não remover.
    
    recomendacoes_svd[user_id] = recs[:K_recs]

print("\n--- Recomendações Finais (ML Puro) ---")
for user, recs in recomendacoes_svd.items():
    print(f"User {user}: {recs}")


--- Recomendações Finais (ML Puro) ---
User 1: ['Luanda', 'Benguela', 'Huíla']
User 2: ['Malanje', 'Cuando Cubango', 'Huambo']
User 3: ['Malanje', 'Cuando Cubango', 'Huambo']


## 6. Conclusões: O Poder da Aprendizagem de Padrões (SVD)

Os resultados do nosso modelo SVD são um sucesso. Eles provam que o ML consegue *aprender* perfis de utilizador complexos e descobrir afinidades (serendipidade) sem que lhe forneçamos quaisquer features.

### 1. A Descoberta de Padrões

O modelo analisou a Matriz Utilizador-Item e **aprendeu** a estrutura de gostos escondida nos nossos dados:

1.  **O "Cluster Mainstream":** O SVD identificou o **User 1** como um perfil distinto, recomendando-lhe `['Luanda', 'Benguela', 'Huíla']`.
2.  **O "Cluster Explorador":** Mais importante, o SVD *descobriu* que o **User 2** e o **User 3** são "vizinhos" (têm gostos semelhantes). Ambos receberam recomendações idênticas: `['Malanje', 'Cuando Cubango', 'Huambo']`.

### 2. A "Previsão" Inteligente

A matriz de previsões (`df_previsoes_svd`) mostra a verdadeira inteligência:

* O **User 2** (que gostou de Malanje/C. Cubango) recebeu um score alto para `Huambo` (**2.44**), um local que ele nunca visitou, mas que o User 3 (o seu "vizinho") adorou.
* O **User 3** (que gostou de Malanje/Huambo) recebeu um score alto para `Cuando Cubango` (**2.44**), um local que ele nunca visitou, mas que o User 2 (o seu "vizinho") adorou.

O modelo de "ML Puro" foi capaz de "preencher os espaços em branco" e recomendar novos destinos relevantes.



### 3. A Fraqueza Crítica (O Ponto Cego)

Apesar deste sucesso, o SVD tem uma fraqueza fatal: o **Problema do "Cold Start" (Arranque a Frio)**.

1.  **Novo Utilizador:** Se um **User 4** se registar amanhã, ele não existe na `df_matrix` original. O modelo SVD é **100% inútil** para ele e não pode fazer previsões.
2.  **Novo Item:** Se adicionarmos **"Moxico"** (uma província sem ratings), ela não existe na `df_matrix`. O SVD **nunca** a poderá recomendar.

### Próximo Passo

Até agora, provámos que:
* O `03_Content_Based` é excelente para o "Cold Start" e `Impacto`, mas é "limitado".
* O `04_Collaborative_Filtering` (SVD) é excelente para *aprender* padrões (serendipidade), mas falha no "Cold Start".

O passo final é óbvio: temos de combinar os dois. Vamos construir o **`05_Hybrid_Model.ipynb`**, que usa o melhor de ambos os mundos para criar um sistema completo.